In [1]:
%load_ext autoreload
%autoreload 2
import math
import torch
import torchvision
import torchvision.transforms as transforms
import torch.utils
import torch.utils.data
import torch.nn as nn
import torchsummary
import torch.nn.functional as F
import numpy as np
from livenet.backend.core import Context
import random
import importlib
import onnx
import livenet
device = "cuda"
# device = "cpu"
torch.set_default_device(device)
from ai_libs.simple_log import LOG



In [2]:
# transform = transforms.Compose(
#     [transforms.ToTensor()
#         ,transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
#      ])
# train=torchvision.datasets.CIFAR10("/home/spometun/datasets/research", train=True,
#                                   download=True, transform=transform)
# test=torchvision.datasets.CIFAR10("/home/spometun/datasets/research", train=False,
#                                   download=True, transform=transform)
#
#
# def get_whole_data(dataset):
#     loader = torch.utils.data.DataLoader(dataset, batch_size=len(dataset))
#     data = next(iter(loader))
#     data[0] = data[0].to(device)
#     data[1] = data[1].to(device)
#     return data
#
# test_whole_data = get_whole_data(test)
# train_whole_data = get_whole_data(train)

# test_x, test_y = test_whole_data
# train_x, train_y = train_whole_data

test_x, test_y = livenet.datasets.get_cifar10_test()
train_x, train_y = livenet.datasets.get_cifar10_train()

test_x = test_x.to(device)
test_y = test_y.to(device)
train_x = train_x.to(device)
train_y = train_y.to(device)



In [19]:


def set_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

set_seed(0)
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.context = Context(self)
        self.max_pool = nn.MaxPool2d(2, 2)
        self.av_pool = nn.AvgPool2d(2, 2)
        self.conv1 = nn.Conv2d(3, 8, 3)
        self.conv2 = nn.Conv2d(8, 8, 3,padding="same")
        self.conv3 = nn.Conv2d(16, 16, 3,padding="same")
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.fc = nn.Linear(8*7*7,10)
        self._alpha = 0.0001

    def forward(self, x):
        # x = F.relu(self.conv1(x))
        # x = self.max_pool(x)
        x = self.av_pool(x)

        x = F.relu(self.conv1(x))
        x = self.max_pool(x)
        # x = self.pool(x)
        # x = F.relu(self.conv3(x))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        # x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        # x = self.fc3(x)
        x=self.fc(x)
        return x

    def internal_loss(self):
        loss = torch.tensor(0.)
        for param in self.parameters():
            if len(param.data.shape) > 1:
                loss += self._alpha * torch.sum(torch.abs(param)) / param.data.numel()
        return loss


network = Net()
torchsummary.summary(network, (3, 32, 32), device=device)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         AvgPool2d-1            [-1, 3, 16, 16]               0
            Conv2d-2            [-1, 8, 14, 14]             224
         MaxPool2d-3              [-1, 8, 7, 7]               0
            Linear-4                   [-1, 10]           3,930
Total params: 4,154
Trainable params: 4,154
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.02
Params size (MB): 0.02
Estimated Total Size (MB): 0.05
----------------------------------------------------------------


In [20]:
batch_size = 256
batch_iterator = livenet.gen_utils.batch_iterator(train_x, train_y, batch_size)
criterion = livenet.nets.criterion_classification_n
optimizer = livenet.nets.create_optimizer(network)
trainer = livenet.net_trainer.NetTrainer(network, batch_iterator, criterion, optimizer, epoch_size=len(train_x) // batch_size)
trainer.adaptive_lr = True
optimizer.learning_rate = 0.01



Torch


In [21]:
#network.learning_rate = 0.0005
trainer.step(100000)


Iˈ0.000 194 2.463+0.000reg params=14 lr=0.01000 <string>:34
Iˈ0.405 389 2.118+0.000reg params=14 lr=0.01000 <string>:34
Iˈ0.912 584 2.034+0.000reg params=14 lr=0.01000 <string>:34
Iˈ1.651 779 1.974+0.000reg params=14 lr=0.01000 <string>:34
Iˈ2.416 974 1.949+0.000reg params=14 lr=0.01000 <string>:34
Iˈ3.232 1169 1.920+0.000reg params=14 lr=0.01000 <string>:34
Iˈ4.136 1364 1.906+0.000reg params=14 lr=0.01000 <string>:34
Iˈ4.983 1559 1.896+0.000reg params=14 lr=0.01000 <string>:34
Iˈ5.746 1754 1.885+0.000reg params=14 lr=0.01000 <string>:34
Iˈ6.491 1949 1.882+0.000reg params=14 lr=0.01000 <string>:34
Iˈ7.339 2144 1.868+0.000reg params=14 lr=0.01000 <string>:34
Iˈ8.049 2339 1.866+0.000reg params=14 lr=0.01000 <string>:34
Iˈ8.754 2534 1.863+0.000reg params=14 lr=0.01000 <string>:34
Iˈ9.458 2729 1.860+0.000reg params=14 lr=0.01000 <string>:34
Iˈ10.164 2924 1.857+0.000reg params=14 lr=0.01000 <string>:34
Iˈ11.009 3119 1.854+0.000reg params=14 lr=0.01000 <string>:34
Iˈ11.912 3314 1.852+0.000re

In [22]:

def calc_accuracy(predictions, labels):
    _, predicted = torch.max(predictions.data, 1)
    labels = labels.cpu().numpy()
    labels = np.squeeze(labels, 1)
    predicted = predicted.cpu().numpy()
    correct = np.sum(predicted == labels)
    total = len(labels)
    accuracy = correct / total
    return accuracy

def get_loss(trainer, x, y):
    with torch.no_grad():
        pred = trainer.network(x)
        loss = trainer.criterion(pred, y)
        loss = loss.cpu().item()
        return loss

with torch.no_grad():
    train_outputs = network(train_x)
    test_outputs = network(test_x)

train_loss = get_loss(trainer, train_x, train_y)
test_loss = get_loss(trainer, test_x, test_y)
LOG(f"loss: train: {train_loss:.3f} test: {test_loss:.3f}")

train_accuracy = calc_accuracy(train_outputs, train_y)
test_accuracy = calc_accuracy(test_outputs, test_y)
LOG(f"accuracy, train: {100 * train_accuracy:.1f}% test: {100 * test_accuracy:.1f}%")

Iˈ0.000 loss: train: 1.606 test: 1.721 
Iˈ0.006 accuracy, train: 61.7% test: 58.7% 


In [13]:
def test_func():
    return livenet.gen_utils.batch_iterator(*test_whole_data, batch_size=256, only_one_epoch=True)

def train_func():
    return livenet.gen_utils.batch_iterator(*train_whole_data, batch_size=256, only_one_epoch=True)
import time

net._alpha = 0.0001

def criterion(input, target):
    return nn.functional.cross_entropy(input, target) / math.log(2)

# optimizer = lib.optimizer.MyOptimizer(net.parameters(), lr=0.01)
t0 = time.time()
for epoch in range(50):
    print(f"{time.time() - t0:.3f} sec")
    t0 = time.time()

    running_loss = 0.0
    for i, data in enumerate(train_func(), 1):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss += net.reg_loss_func()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        n_observe = 4 * 195

        # print statistics
        running_loss += loss.item()
    print(f'[{epoch + 1}, {i:5d}] running loss: {running_loss / i:.2f}')
    running_loss = 0.0
    calc_stats(net, train_whole_data)
    calc_stats(net, test_whole_data)
    #lr schedule
    if True:
        observed = np.array(losses[-n_observe:])
        av1 = np.average(observed[:len(observed) // 2])
        av2 = np.average(observed[len(observed) // 2:])
        print(f"av1={av1:.4f} av2={av2:.4f}")
        slope, pvalue = livenet.stat_utils.get_slope_and_pvalue(losses[-n_observe:])
        print(f"slope={slope:.1e} pvalue={pvalue:.1e} lr={optimizer.param_groups[0]['lr']}")
        if slope >= 0.0:
            optimizer.param_groups[0]["lr"] /= 1.4
            print(f"reduced lr to {optimizer.param_groups[0]['lr']:.2e}")

print('Finished Training')


NameError: name 'net' is not defined